In [0]:
import sys
from pyspark.sql.functions import when
from pyspark.sql import SparkSession


In [0]:
args = getResolvedOptions(sys.argv,['s3_target_path_key','s3_target_path_bucket'])
bucket = args['s3_target_path_key']
fileName = args['s3_target_path_bucket']

print(bucket, fileName)

In [0]:
spark = SparkSession.builder.appName("CDC1").getOrCreate()
inputFilePath = f"s3a://{bucket}/{fileName}"
finalFilePath = f"s3a://cdc-output-pyspark/output"

In [0]:
if "LOAD" in fileName:
    df0 = spark.read.csv(inputFilePath)
    df0 = df0.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2","Location")
    df0.write.mode("overwrite").csv(finalFilePath)
else:
    u_df = spark.read.csv(inputFilePath)
    u_df = u_df.withColumnRenamed("_c0","action").withColumnRenamed("_c1","id").withColumnRenamed("_c2","FullName").withColumnRenamed("_c3","Location")
    r_df = spark.read.csv(finalFilePath)
    r_df = r_df.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2","Location")
    
    for row in u_df.collect():
        if row["action"] == "U":
            r_df = r_df.withColumn("FullName", when(r_df["id"] == row["id"], row["FullName"]).otherwise(r_df["FullName"]))
            r_df = r_df.withColumn("Location", when(r_df["id"] == row["id"], row["Location"]).otherwise(r_df["Location"]))
        
        if row["action"] == "I":
            newrow = [list(row)[1:]]
            columns = ['id', 'FullName','Location']
            inserted_df = spark.createDataFrame(newrow, columns)
            r_df = r_df.union(newrow)
        
        if row["action"] == "D":
            r_df = r_df.filter(r_df.id != row["id"])
    
    r_df.write.mode("overwrite").csv(finalFilePath)
    
    

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-4450083387114013> in <cell line: 1>()
----> 1 if "LOAD" in fileName:
      2     df0 = spark.read.csv(inputFilePath)
      3     df0 = df0.withColumnRenamed("_c0","id").withColumnRenamed("_c1","FullName").withColumnRenamed("_c2","Location")
      4     df0.write.mode("overwrite").csv(finalFilePath)
      5 else:

NameError: name 'fileName' is not defined